In [1]:

import calendar
months = list(map(lambda x:x.lower(),calendar.month_name))[1:]
import pandas as pd
from pandas import DataFrame

import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import matplotlib.ticker as mtick

import matplotlib.pylab as pylab
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 10),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)

import os
project_folder =os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
import warnings
warnings.filterwarnings(action='ignore')
from IPython.display import display
from IPython.display import Markdown
from IPython.display import display_html




In [2]:
# That's not part of the procedure
pd.read_csv(f'{project_folder}/data/process/new_data.csv').reset_index().iloc[0].to_csv('output_2/fileds_to_work_on.csv')

In [2]:
# show the questions
data_questions = pd.read_csv(f'{project_folder}/data/process/new_data.csv', skiprows=[2]).reset_index().iloc[0]
data_questions

index                                                          0
StartDate                                             Start Date
EndDate                                                 End Date
Status                                             Response Type
IPAddress                                             IP Address
                                     ...                        
Q38            What is the highest level of education you hav...
Q39            What do you consider your main ethnic origin o...
Q39_10_TEXT    What do you consider your main ethnic origin o...
Q40            Where did you hear about this survey? - Select...
Q40_3_TEXT     Where did you hear about this survey? - Other ...
Name: 0, Length: 75, dtype: object

#### Data Preparation

In [3]:
data_general = pd.read_csv(f'{project_folder}/data/process/final_geo_coding.csv', skiprows=[1, 2]).reset_index()
# Read dictionary of relevant questions
questions_to_examine=pd.read_csv(f'output_2/fileds_to_work_on_updated.csv')
rel_questions_dic = dict(zip(questions_to_examine['Number'], questions_to_examine['Abbreviation ']))
field_to_work_on = 'index'

In [4]:
special_questions = ['Q26', 'Q27', 'Q32', 'Q33']


In [5]:
questions_to_examine_1  = questions_to_examine.set_index('Number')

In [63]:
res_reg = {}
for question in rel_questions_dic.items():
    question_num = question[0]
    if question_num in special_questions:
        continue
    gb_data= data_general.groupby(question_num).count()[field_to_work_on].rename('count').reset_index()
    sumy = gb_data['count'].sum()
    gb_data['per'] = round(gb_data['count']/sumy *100,0)
    gb_data.sort_values(by='per',ascending=False,inplace=True)



    res_reg[question_num] = gb_data
    # Draw the resuils:
    # gb_data['per'].plot(kind='bar',title=question[1])
    # plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0,decimals=0))
    # plt.show()
res_reg

{'Q25':                                                  Q25  count   per
 2  ​Yes, and I replaced it with exactly what was ...    828  46.0
 0                                                 No    558  31.0
 1     Yes, but the replacement was something cheaper    426  24.0,
 'Q28_1':                     Q28_1  count   per
 1         1-3 days a week    593  34.0
 2   4 or more days a week    518  30.0
 0        1-3 days a month    307  18.0
 5                   Never    154   9.0
 4  Less than once a month    143   8.0
 3     Don't know/not sure     16   1.0,
 'Q28_2':                     Q28_2  count   per
 5                   Never    433  25.0
 1         1-3 days a week    377  22.0
 2   4 or more days a week    358  21.0
 0        1-3 days a month    277  16.0
 4  Less than once a month    239  14.0
 3     Don't know/not sure     16   1.0,
 'Q28_3':                     Q28_3  count   per
 1         1-3 days a week    587  34.0
 2   4 or more days a week    548  32.0
 0        1-3 d

#### Analysis for Q26

In [11]:
data_general['Q26'] = data_general['Q26'].apply(lambda x: str(x).rstrip().lstrip())
gb_data_0= data_general.groupby('Q26').count()[field_to_work_on].sort_values(ascending=False)
# Values to delete
null_values =['nan','N/a','Na','na','None','-','Months']
gb_data = gb_data_0[~gb_data_0.index.isin(null_values)]

In [10]:
def calc_statistics(class_data:DataFrame):
    r"""
    the class count the number of respondents for the number of days
    :param class_data:
    :return: list of date groupby the number of days, statistics and count and percentage of those with doesn't recover/replace/fix atc. their bicycles
    """
    # remove records without number of days
    num_of_days = class_data['num_of_days']
    final_data = class_data[(num_of_days!='') & (num_of_days !=-1)]
    # group by number of days
    mygroupby = final_data.groupby('num_of_days').sum().sort_values(by = 'count',ascending=False)

     # flat @mygroupby to calculate statistic
    l= [[item[0]]*item[1]['count'] for item in mygroupby.iterrows()]
    flat_list = [item for sublist in l for item in sublist]
    df = DataFrame(flat_list)


    # the number of -1
    no_data = class_data[num_of_days ==-1].sum()['count']
    no_dat_dic = (no_data ,no_data/ (len(flat_list)+no_data)*100)
    return mygroupby, df.describe(),{'no_action_yet':no_dat_dic},df


In [13]:
from qu_26_27 import InterpretTime
time_q26 = InterpretTime(gb_data)
for res in time_q26.new_gb_data.iterrows():
    time_q26.main_part(res)
res_26 = calc_statistics(time_q26.new_gb_data)

#### Analysis for Q27

In [15]:
gb_data_first = data_general.groupby('Q27').count()[field_to_work_on].sort_values(ascending=False)
gb_data_27 = gb_data_first.dropna()
time_q27 = InterpretTime(gb_data_27)
for res in time_q27.new_gb_data.iterrows():
    time_q27.main_part(res)
res_q27 = calc_statistics(time_q27.new_gb_data)

#### Code for Q32 and Q33 which is multiple choices questions

In [38]:
res32_33=[]
for question_num in special_questions[-2:]:
    print(question_num)
    # Read choice options
    choice_df = pd.read_csv(f'output_2/{question_num}.csv',encoding='cp1252').set_index('question')
    # grpoupby data by the responses to the question_num
    gb_data= data_general.groupby(question_num).count()[field_to_work_on].sort_values(ascending=False).reset_index()
    # Go over each separate response
    for  item in choice_df.iterrows():
        question = item[0]
        # search the current response in the multiple choices responses
        for row_0 in gb_data.iterrows():
            row = row_0[1]
            if question in row[question_num]:
                # if the response was found add the number of respondants that use that respomse as part on 1 or combination of choices
                choice_df.loc[question]+=row[field_to_work_on]
    # calculate percentage
    sumy = choice_df.sum()
    choice_df['per'] = round(choice_df/sumy *100,0)
    choice_df.sort_values(by='per',ascending=False,inplace=True)
    res32_33.append(choice_df)
    # Plot results
    # choice_df['per'].plot(kind='bar',title=rel_questions_dic[question_num])

    # plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0,decimals=0))
    # plt.show()

Q32
Q33


In [34]:
html_table = questions_to_examine_1.to_html()
text ="<h4>The following questions will be the main focus of our analysis. We will begin by looking at each question individually.</h4>"
display(Markdown(text))
display_html(html_table, raw=True)
questions_to_examine_1

<h4>The following questions will be the main focus of our analysis. We will begin by looking at each question individually.</h4>

,Question,Abbreviation
Number,,
Q25,Did you replace the stolen bicycle or bicycle part?,Is replaced
Q26,How many days did it take to recover or replace your stolen bicycle or part?,Time replaced
Q27,"After recovering your bike, how many additional days did it take before your bicycle was repaired and rideable?",Time rideable
Q28_1,"Over the past year, how often did you cycle (with any bicycle) during each season? - Fall (Sept, Oct, Nov)",Fall habit
Q28_2,"Over the past year, how often did you cycle (with any bicycle) during each season? - Winter (Dec, Jan, Feb)",Winter habit
Q28_3,"Over the past year, how often did you cycle (with any bicycle) during each season? - Spring (Mar, Apr, May)",Spring habit
Q28_4,"Over the past year, how often did you cycle (with any bicycle) during each season? - Summer (Jun, Jul, Aug)",Summer habit
Q29,"What was the main purpose of the trips you made with the stolen/damaged bicycle? Measured in highest number or trips, not distance cycled",Purpose trip
Q30,"Thinking about the trips you made with the stolen/damaged bicycle, what transportation mode did you most often use to make those trips while your bicycle was unavailable?",Mode switch


,Question,Abbreviation
Number,,
Q25,Did you replace the stolen bicycle or bicycle ...,Is replaced
Q26,How many days did it take to recover or replac...,Time replaced
Q27,"After recovering your bike, how many additiona...",Time rideable
Q28_1,"Over the past year, how often did you cycle (w...",Fall habit
Q28_2,"Over the past year, how often did you cycle (w...",Winter habit
Q28_3,"Over the past year, how often did you cycle (w...",Spring habit
Q28_4,"Over the past year, how often did you cycle (w...",Summer habit
Q29,What was the main purpose of the trips you mad...,Purpose trip
Q30,Thinking about the trips you made with the sto...,Mode switch


In [66]:
q_num ='Q25'
q= questions_to_examine_1.loc[q_num][0]
res_25 = res_reg[q_num]
more_ex = int(res_25[res_25['Q25'] =='\u200bYes, and I replaced it with exactly what was stolen or something more expensive ']['per'].values[0])
cheaper = int(res_25[res_25['Q25'] =='Yes, but the replacement was something cheaper']['per'].values[0])
no= int(res_25[res_25['Q25'] =='No']['per'].values[0])
text =f"{q_num} - {q}"
display(Markdown(text))
text = f"<h4>Results</h4><ul><li>{no}% did not replace their stolen bicycle.</li><li> {cheaper}% did replace with cheaper bicycle.</li><li> {more_ex}%  with more expensive bicycle.</li></ul>"
display(Markdown(text))
text = "<h4>Next analysis</h4><ul> <li>cross validation with demographic</li> <li>cross validation with bicycle value</li></ul>"
display(Markdown(text))

Q25 - Did you replace the stolen bicycle or bicycle part?

<h4>Results</h4><ul><li>31% did not replace their stolen bicycle.</li><li> 24% did replace with cheaper bicycle.</li><li> 46%  with more expensive bicycle.</li></ul>

<h4>Next analysis</h4><ul> <li>cross validation with demographic</li> <li>cross validation with bicycle value</li></ul>

In [27]:
q_num = 'Q26'
q= questions_to_examine_1.loc[q_num][0]
res = res_26
flat_data = res[-1]
day_25 = int(res[1].loc['25%'][0])
day_50 = int(res[1].loc['50%'][0])
more_365  = len(flat_data[flat_data[0]>365])
more_385_percentage = int(round(more_365/len(flat_data)*100,0))
max= int(res[1].loc['max'][0])
no_recoverd_replace =int(round(list(res[2].values())[0][1]))
text =f"{q_num} - {q}"
display(Markdown(text))
text = f"<h4>Results</h4><ul><li>25% recovered or replaced their bicycles within {day_25} days and 50% within {day_50} days.</li>  <li>{more_385_percentage}% are recovered or replaced over 1 year.</li> <li>The longest time  reported to recovered/replaced was {max} days.</li><li>Among all the participants that answers to this question {no_recoverd_replace}% have not recovered/replaces their bicycles.</ul>"
display(Markdown(text))
text = "<h4>Next analysis</h4><ul> <li>compare the results between group members who were replaced and those who recovered. Are there any statistical differences between the two groups? </li> <li>cross validation with demographic</li><li>cross validation with bike characteristics</li></ul>"
display(Markdown(text))


Q26 - How many days did it take to recover or replace your stolen bicycle or part?

<h4>Results</h4><ul><li>25% recovered or replaced their bicycles within 20 days and 50% within 49 days.</li>  <li>4% are recovered or replaced over 1 year.</li> <li>The longest time  reported to recovered/replaced was 1460 days.</li><li>Among all the participants that answers to this question 5% have not recovered/replaces their bicycles.</ul>

<h4>Next analysis</h4><ul> <li>compare the results between group members who were replaced and those who recovered. Are there any statistical differences between the two groups? </li> <li>cross validation with demographic</li><li>cross validation with bike characteristics</li></ul>

In [109]:
q_num = 'Q27'
res = res_q27
q= questions_to_examine_1.loc[q_num][0]
flat_data = res[-1]
day_25 = int(res[1].loc['25%'][0])
day_50 = int(res[1].loc['50%'][0])
more_30  = len(flat_data[flat_data[0]>30])
more_30_percentage = int(round(more_30/len(flat_data)*100,0))
max= int(res[1].loc['max'][0])
no_recoverd_replace =int(round(list(res[2].values())[0][1]))

text =f"{q_num} - {q}"
display(Markdown(text))
text = f"<h4>Results</h4><ul><li>25% repaired their bicycles within {day_25} days and 50% within {day_50} days.</li>  <li> {more_30_percentage}% are repaired after more then a month.</li> <li>The longest time  reported to repair was {max} days.</li> <li>Among all the participants that answers to this question {no_recoverd_replace}% have not repaired their bicycles.</li></ul>"
display(Markdown(text))
text = "<h4>Next analysis</h4><ul> <li>cross validation with bike condition upon recovery</li> <li>cross validation with demographic</li><li>cross validation with bike characteristics</li></ul>"
display(Markdown(text))



Q27 - After recovering your bike, how many additional days did it take before your bicycle was repaired and rideable?

<h4>Results</h4><ul><li>25% repaired their bicycles within 7 days and 50% within 14 days.</li>  <li> 28% are repaired after more then a month.</li> <li>The longest time  reported to repair was 360 days.</li> <li>Among all the participants that answers to this question 3% have not repaired their bicycles.</li></ul>

<h4>Next analysis</h4><ul> <li>cross validation with bike condition upon recovery</li> <li>cross validation with demographic</li><li>cross validation with bike characteristics</li></ul>

In [111]:
frequency= 'frequency'
seasons = {1:'Fall', 2:'Winter', 3:'Spring', 4:'Summer'} # list of seasons
data_to_use  =[res_reg[f'Q28_{i}'].rename(columns={'per':seasons[i]}) for i in range(1,5)]
data_to_use_1 =[data.rename(columns= {data.columns[0]:frequency}).drop('count', axis=1).set_index(frequency) for data in data_to_use]
seasons_pd= pd.concat(data_to_use_1,axis=1)

frequency_dic={'1-3 days a week':1, '4 or more days a week':0, '1-3 days a month':2, 'Never':4,
       'Less than once a month':3, "Don't know/not sure":5}
for_sort = 'for_sort'
seasons_pd[for_sort] = frequency_dic
seasons_pd.sort_values(for_sort).drop(for_sort,axis=1).T

q_num = 'Q28'
q =f"Over the past year, how often did you cycle"
text =f"{q_num} - {q}"
display(Markdown(text))
text = f"<h4>Results</h4><ul><li>The seasons have huge effect on the cycling habit.</li></ul>"
display(Markdown(text))
text = "<h4>Next analysis</h4><ul> <li>Find group of cyclists with same habit</li> <li>for those groups cross validation with demographic</li><li>with bike characteristics </li><li> with other questions behaviour</li> <li>between cities</li></ul>"
display(Markdown(text))


frequency,4 or more days a week,1-3 days a week,1-3 days a month,Less than once a month,Never,Don't know/not sure
Fall,30.0,34.0,18.0,8.0,9.0,1.0
Winter,21.0,22.0,16.0,14.0,25.0,1.0
Spring,32.0,34.0,18.0,6.0,9.0,1.0
Summer,41.0,35.0,11.0,5.0,7.0,1.0


Q28 - Over the past year, how often did you cycle

<h4>Results</h4><ul><li>The seasons have huge effect on the cycling habit.</li></ul>

<h4>Next analysis</h4><ul> <li>Find group of cyclists with same habit</li> <li>for those groups cross validation with demographic</li><li>with bike characteristics </li><li> with other questions behaviour</li> <li>between cities</li></ul>

In [113]:
q_num ='Q29'
res_reg[q_num]
q= questions_to_examine_1.loc[q_num][0]
q

,Q29,count,per
2,"Recreation or exercise (e.g., cycling in your ...",721,42.0
3,"Transportation (e.g., commuting to school or w...",542,31.0
1,"Equal mix of recreation, exercise, and transpo...",455,26.0
0,Don’t know/not sure,10,1.0


'What was the main purpose of the trips you made with the stolen/damaged bicycle? Measured in highest number or trips, not distance cycled'

In [67]:
q_num ='Q32'
q= questions_to_examine_1.loc[q_num][0]
most_pop = res32_33[0].iloc[0]
text =f"{q_num} - {q}"
display(Markdown(text))
text = f"<h4>Results</h4><ul><li>the answer with the highest  frequency is:</li></ul>"
display(Markdown(text))
most_pop
q_num = 'Q33'
q= questions_to_examine_1.loc[q_num][0]
most_pop = res32_33[1].iloc[0]
text =f"{q_num} - {q}"
display(Markdown(text))
len_all =len(data_general)
percentage_of_ans = int(round((len_all- data_general[q_num].isna().sum())/len_all*100))
text = f"<h4>Results</h4><ul><li>only {percentage_of_ans}% answer this question.</li><li>the answer with the highest  frequency is :</li></ul>"
display(Markdown(text))
most_pop

text = "<h4>Next analysis</h4><ul> <li>group choices</li> <li>cross validation with demographic</li><li>cross validation with bike characteristics</li></ul>"
display(Markdown(text))

Q32 - What were the main reasons you cycled less often or not at all? (Select up to 3) - Selected Choice

<h4>Results</h4><ul><li>the answer with the highest  frequency is:</li></ul>

count    307.0
per       25.0
Name: I couldn’t recover my bicycle and didn’t replace it due to cost or other reason, dtype: float64

Q33 - What were the main reasons you cycled more often? (Select up to 3) - Selected Choice

<h4>Results</h4><ul><li>only 6% answer this question.</li><li>the answer with the highest  frequency is :</li></ul>

count    49.0
per      26.0
Name: My new bicycle was more capable, comfortable to ride, or better suited to my travel needs, dtype: float64

<h4>Next analysis</h4><ul> <li>group choices</li> <li>cross validation with demographic</li><li>cross validation with bike characteristics</li></ul>

In [99]:
list(seasons.values())

['Fall', 'Winter', 'Spring', 'Summer']